# Compute and Display Solomon Islands geometry from tiles

In [1]:
import functools
import os

import geopandas as gpd
import ipyleaflet as ipyl
import json
from tqdm import tqdm



In [2]:
# Directory where tile .zip files from gs://ei-embeddings/tiles/solomon are stored 
solomon_tile_dir = "/Users/ben/Downloads/solomon/"
files = os.listdir(solomon_tile_dir)

In [3]:
# Load tile geometries to list
tiles = []
for file in tqdm(files):
    gdf = gpd.read_file(os.path.join(solomon_tile_dir,file))
    
    # Make sure the projection is consistent
    if gdf.crs.name != 'WGS 84 / UTM zone 57S':
        gdf = gdf.to_crs("EPSG:32757")
    
    tiles.append(gdf.unary_union)
     

100%|█████████████████████████████████████████████████████████████████████████████████████| 108/108 [00:52<00:00,  2.07it/s]


In [4]:
# Get union of all geometries
total_geom = functools.reduce(lambda x, y: x.union(y), tiles)

In [5]:
# Load to GeoSeries and change CRS from UTM 57S to WGS84
gs = gpd.GeoSeries(total_geom).set_crs(epsg = 32757).to_crs(epsg = 4326)

# Save geojson (optional)
#gs.to_file("/Users/ben/Downloads/solomon.geojson")

In [6]:
def build_region_layer(geometry, weight: int) -> ipyl.GeoJSON:
    """
    Build a layer for a region.
    """
    region_layer = ipyl.GeoJSON(
        name="region",
        data=json.loads(geometry.to_json()),
        style={
            'color': 'blue',
            'opacity': 1,
            'fillOpacity': 0,
            'weight': weight
        }
    )
    return region_layer

In [7]:
# Get layer for display
layer = build_region_layer(gs, weight=5)

In [8]:
# Display on map
center = [-9, 161]
zoom = 6

m = ipyl.Map( center=center, zoom=zoom)
m.add_layer(layer)
m

Map(center=[-9, 161], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_te…